In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
import pandas as pd

# Read the Excel file into a pandas DataFrame
df_products = pd.read_excel("730-Raw Data V1.xlsx")
df_products

,Food product,Chocolate Cookie,Butter Cookie,Cheese Cake,Coconut Brownie,Red Velvet,Muffins,Brownie,Donuts,Fruit Bun,Cheese Pizza,Cheese Burger,Sugarless Cookie,Dark Chocolate Pie
0,Sugar (grams/unit),10,7,8,6,9,8,11,10,15,1,3,0,0
1,Wheat (grams/unit),20,25,15,15,22,10,24,20,30,45,35,21,25
2,Salt (grams/unit),3,2,6,2,0,2,1,4,5,11,10,2,3
3,Baking Soda (grams/unit),1,1,3,1,3,1,2,2,2,3,3,2,1
4,Chocolate chips (grams/unit),15,0,0,4,0,6,25,0,0,0,0,0,15
5,Cheese (ml/unit),0,0,22,0,0,0,0,0,0,43,31,0,0
6,Eggs,1,1,2,1,2,1,3,3,1,1,2,1,1
7,Fat (ml/unit),30,55,29,31,30,15,34,15,25,24,22,35,18
8,Minimum Production,40,40,10,10,20,15,10,15,10,5,7,15,10


In [ ]:
df_profits = pd.read_excel("730-Raw Data V1.xlsx", sheet_name='Profits')
df_profits

,Food product,Chocolate Cookie,Butter Cookie,Cheese Cake,Coconut Brownie,Red Velvet,Muffins,Brownie,Donuts,Fruit Bun,Cheese Pizza,Cheese Burger,Sugarless Cookie,Dark Chocolate Pie
0,Cost/unit,3.5,1.5,1.5,2.5,2,2.0,1.8,1.5,1.5,3,2.5,2.5,2.5
1,Revenue/unit,7.0,4.0,5.0,6.0,4,3.5,5.5,3.0,2.5,6,6.0,5.0,4.0
2,Profit/unit,3.5,2.5,3.5,3.5,2,1.5,3.7,1.5,1.0,3,3.5,2.5,1.5


In [ ]:
df_stocks = pd.read_excel("730-Raw Data V1.xlsx", sheet_name='Stock')
df_stocks

,Sugar (grams),Wheat (grams),Salt (grams),Baking Soda (grams),Chocolate chips (grams),Cheese (ml),Eggs (units),Fat (ml)
0,10000,12000,8000,1000,4500,4500,650,7000


In [ ]:
#convert to list of lists
products = df_products.iloc[:,1:].values.tolist()
profits = df_profits.iloc[:,1:].values.tolist()
stocks = df_stocks.iloc[:,0:].values.tolist()

In [ ]:
print(products[8])

[40, 40, 10, 10, 20, 15, 10, 15, 10, 5, 7, 15, 10]


In [ ]:
#declare a concrete model
model = ConcreteModel()

num_products = 13 #use i to index the products, this is the first index
num_ing = 8 #use j to index the ingredients, this is the second index

#declare the decision variables
model.x = Var(range(num_products), domain=NonNegativeReals)
model.y = Var(range(num_products), domain=Binary)  # Binary decision variable

# Objective Function: Maximize Total Profit
model.total_profit = Objective(expr=sum(profits[2][i] * model.x[i] for i in range(num_products)), sense=maximize)

In [ ]:
# Additional Constraint: Sugar Used <= Sugar Available
model.sugar_constraint = Constraint(expr=sum(products[0][i] * model.x[i] for i in range(num_products)) <= stocks[0][0])

# Additional Constraint: Wheat Used <= Wheat Available
model.wheat_used_constraint = Constraint(expr=sum(products[1][i] * model.x[i] for i in range(num_products)) <= stocks[0][1])

# Additional Constraint: Salt Used <= Salt Available
model.salt_used_constraint = Constraint(expr=sum(products[2][i] * model.x[i] for i in range(num_products)) <= stocks[0][2])

# Additional Constraint: Baking Soda Used <= Baking Soda Available
model.baking_soda_used_constraint = Constraint(expr=sum(products[3][i] * model.x[i] for i in range(num_products)) <= stocks[0][3])

# Additional Constraint: Chocolate Chips Used <= Chocolate Chips Available
model.chocolate_chips_used_constraint = Constraint(expr=sum(products[4][i] * model.x[i] for i in range(num_products)) <= stocks[0][4])

# Additional Constraint: Cheese Used <= Cheese Available
model.cheese_used_constraint = Constraint(expr=sum(products[5][i] * model.x[i] for i in range(num_products)) <= stocks[0][5])

# Additional Constraint: Eggs Used <= Eggs Available
model.eggs_used_constraint = Constraint(expr=sum(products[6][i] * model.x[i] for i in range(num_products)) <= stocks[0][6])

# Additional Constraint: Fat Used <= Fat Available
model.fat_used_constraint = Constraint(expr=sum(products[7][i] * model.x[i] for i in range(num_products)) <= stocks[0][7])

In [ ]:
bigM = 1000000 # Use a large constant as an upper bound
#Minimum production constraint
min_prod_constraint_expr = sum(model.x[i] for i in range(num_products)) >= sum(model.y[i] * products[8][i] for i in range(num_products))
min_prod_constraint = Constraint(expr=min_prod_constraint_expr)

#Maximum production constraint
max_prod_constraint_expr = sum(model.x[i] for i in range(num_products)) <= sum(model.y[i] * bigM for i in range(num_products))
max_prod_constraint = Constraint(expr=max_prod_constraint_expr)

In [ ]:
model.pprint()

2 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   13 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    y_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   13 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

2 Var Declarations
    x : Size=13, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : False :  True : NonNegativeReals
          6 :     0 :  None :  None : False :  True : NonNegativeReals
          

In [ ]:
#solve the model
opt = SolverFactory('cbc')
opt.solve(model, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmpxzb_tmli.pyomo.lp -stat=1 -solve -solu /tmp/tmpxzb_tmli.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 8 (0) rows, 13 (0) columns and 83 (0) elements
Statistics for presolved model


Problem has 8 rows, 13 columns (13 with objective) and 83 elements
Column breakdown:
13 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
8 of type L other, 0 of type Range 0.0->1.0, 0 of typ

{'Problem': [{'Name': 'unknown', 'Lower bound': 952.1505376, 'Upper bound': 952.1505376, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 13, 'Number of nonzeros': 13, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 3}}, 'Error rc': 0, 'Time': 0.06971859931945801}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
# Print the objective value
print("\nObjective Value: ", model.total_profit())

# Print the decision variables
print("\nDecision Variables:")
for i in range(num_products):
    print(f"Product {i + 1}: Quantity = {model.x[i].value}, Binary Decision = {model.y[i].value}")



Objective Value:  952.150535

Decision Variables:
Product 1: Quantity = 126.88172, Binary Decision = None
Product 2: Quantity = 0.0, Binary Decision = None
Product 3: Quantity = 0.0, Binary Decision = None
Product 4: Quantity = 0.0, Binary Decision = None
Product 5: Quantity = 0.0, Binary Decision = None
Product 6: Quantity = 0.0, Binary Decision = None
Product 7: Quantity = 0.0, Binary Decision = None
Product 8: Quantity = 0.0, Binary Decision = None
Product 9: Quantity = 0.0, Binary Decision = None
Product 10: Quantity = 0.0, Binary Decision = None
Product 11: Quantity = 145.16129, Binary Decision = None
Product 12: Quantity = 0.0, Binary Decision = None
Product 13: Quantity = 0.0, Binary Decision = None


***Therefore, the maximum profit per day is $ 952.15. We need to make 127 units of product 1 (Chocolate Cookie) and 146 units of product 11 (Cheese Burger) to get the maximum profit.***